In [46]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

dtr = pd.read_csv('dataset/train.csv')
dts = pd.read_csv('dataset/test.csv')

In [32]:
print(list(set(df['gender'])))

[0, 1]


In [47]:
for column in dtr.columns:
    if dtr.dtypes[column] == 'int64':
        continue
    for df in [dtr, dts]:
        df.replace(to_replace={value: index for index, value in enumerate(set(df[column]))}, inplace=True)

/tmp/ipykernel_1121129/2411171329.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={value: index for index, value in enumerate(set(df[column]))}, inplace=True)
/tmp/ipykernel_1121129/2411171329.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={value: index for index, value in enumerate(set(df[column]))}, inplace=True)
/tmp/ipykernel_1121129/2411171329.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old beha

In [79]:
income = 'income_>50K'

train_part = dtr[:20000]
test_part = dtr[20000:]

Y_train = train_part[income]
X_train = train_part.drop(income, axis=1)
Y_test = test_part[income]
X_test = test_part.drop(income, axis=1)

# print(y_train)
# print(X_train)
# print(X_test)

In [73]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)

knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)

decision_tree = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    # 'max_depth': [None, 5, 10, 15],
    # 'min_samples_split': [2, 5, 10],
    # 'min_samples_leaf': [55, 60, 65]
}

grid_search = GridSearchCV(decision_tree, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Наилучшие параметры:", grid_search.best_params_)
print("Лучшая оценка:", grid_search.best_score_)

Наилучшие параметры: {'criterion': 'log_loss'}
Лучшая оценка: 0.816866521399947


In [71]:
knn = KNeighborsClassifier()
param_grid = {
    'n_jobs': [-1],
    'leaf_size': [2, 3, 4],
    # 'n_neighbors': [20, 25, 30],
    # 'weights': ['uniform', 'distance']
}

grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, Y_train)

print("Наилучшие параметры:", grid_search.best_params_)
print("Лучшая оценка:", grid_search.best_score_)

Наилучшие параметры: {'leaf_size': 3, 'n_jobs': -1}
Лучшая оценка: 0.776031156854625


In [80]:
decision_tree = DecisionTreeClassifier(
    criterion = 'log_loss',
    max_depth = 10,
    min_samples_leaf = 60,
    min_samples_split = 5
)
knn = KNeighborsClassifier(
    leaf_size = 3,
    n_jobs = -1,
    n_neighbors = 25, 
    weights = 'uniform'
)
decision_tree.fit(X_train, Y_train)
knn.fit(X_train, Y_train)


y_pred_DT_train = decision_tree.predict(X_train)
y_pred_knn_train = knn.predict(X_train)

y_pred_decision_tree = decision_tree.predict(X_test)
y_pred_knn = knn.predict(X_test)

# print("Результаты предсказания модели дерева решений:", y_pred_decision_tree)
# print("Результаты предсказания модели k-ближайших соседей:", y_pred_knn)

accuracy_decision_tree = accuracy_score(Y_train, y_pred_DT_train)
accuracy_knn = accuracy_score(Y_train, y_pred_knn_train)

print("Точность модели дерева решений:", accuracy_decision_tree)
print("Точность модели k-ближайших соседей:", accuracy_knn)

Точность модели дерева решений: 0.8575
Точность модели k-ближайших соседей: 0.8012


In [81]:
from sklearn.metrics import confusion_matrix

cm_decision_tree = confusion_matrix(Y_test, y_pred_decision_tree)
cm_knn = confusion_matrix(Y_test, y_pred_knn)

print("Матрица ошибок для модели дерева решений:")
print(cm_decision_tree)

print("Матрица ошибок для модели k-ближайших соседей:")
print(cm_knn)

Матрица ошибок для модели дерева решений:
[[17120  1129]
 [ 2372  3336]]
Матрица ошибок для модели k-ближайших соседей:
[[18115   134]
 [ 4682  1026]]
